# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [18]:
import numpy as np
import pandas as pd

# For splitting data, hyperparam tuning, and evaluation
from sklearn.model_selection import train_test_split, GridSearchCV

# Transformers & pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Metrics (with custom F1 for pos_label='yes')
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, make_scorer

# Classification models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Display settings for Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# pd.options.display.float_format = '{:.2f}'.format  # Optional numeric formatting

In [19]:
'''
Based on the summary of the variables we can see the following important notes:
1) 'unknown' values for several columns including 
    job, material, education, defualt, housing, loan, duration

2) Duration - there can be calls that were zero minutes meaning y was no. this input should only be included for 
    benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

3) pdays - 999 means client was not previously contacted. 

   Input variables:
   # bank client data:
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   5 - default: has credit in default? (categorical: "no","yes","unknown")
   6 - housing: has housing loan? (categorical: "no","yes","unknown")
   7 - loan: has personal loan? (categorical: "no","yes","unknown")
   # related with the last contact of the current campaign:
   8 - contact: contact communication type (categorical: "cellular","telephone") 
   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
  11 - duration: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. 
        Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
   # other attributes:
  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
  14 - previous: number of contacts performed before this campaign and for this client (numeric)
  15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
   # social and economic context attributes
  16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
  17 - cons.price.idx: consumer price index - monthly indicator (numeric)     
  18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     
  19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
  20 - nr.employed: number of employees - quarterly indicator (numeric)

  Output variable (desired target):
  21 - y - has the client subscribed a term deposit? (binary: "yes","no")
'''

'\nBased on the summary of the variables we can see the following important notes:\n1) \'unknown\' values for several columns including \n    job, material, education, defualt, housing, loan, duration\n\n2) Duration - there can be calls that were zero minutes meaning y was no. this input should only be included for \n    benchmark purposes and should be discarded if the intention is to have a realistic predictive model.\n\n3) pdays - 999 means client was not previously contacted. \n\n   Input variables:\n   # bank client data:\n   1 - age (numeric)\n   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")\n   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)\n   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [20]:
df = pd.read_csv(
    '/Users/jimlopez/Desktop/AI Bootcamp/Practical Application Assignment 17.1- Comparing Classifiers/bank-additional/bank-additional-full.csv',
    sep=';'
)

print("First top 15 rows:\n", df.head(15))
print("\nDescription of all columns:\n", df.describe(include='all'))
# Observed: 'unknown' values for certain columns (education, default, etc.)

print("\nNon-null counts and data types for each column:")
print(df.info())


First top 15 rows:
     age          job   marital            education  default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y
0    56    housemaid   married             basic.4y       no      no   no  telephone   may         mon       261         1    999         0  nonexistent           1.1          93.994          -36.4      4.857       5191.0  no
1    57     services   married          high.school  unknown      no   no  telephone   may         mon       149         1    999         0  nonexistent           1.1          93.994          -36.4      4.857       5191.0  no
2    37     services   married          high.school       no     yes   no  telephone   may         mon       226         1    999         0  nonexistent           1.1          93.994          -36.4      4.857       5191.0  no
3    40       admin.   married             basic.6y       no      no   no  t

In [21]:

# Summaries about distribution of target
print(f"\nTarget Distribution:\n{df['y'].value_counts()}")
print(f"\nNull check per column:\n{df.isnull().sum()}")
# We see no true NaN missing cells, but 'unknown' placeholders exist for some columns.



Target Distribution:
y
no     36548
yes     4640
Name: count, dtype: int64

Null check per column:
age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [22]:
# Duration can leak info: if duration=0, often 'no'. 
# We'll keep it for now, but we can drop it if we want a real-time model.

USE_DURATION = True
if not USE_DURATION:
    df.drop('duration', axis=1, inplace=True)
    print("Excluding 'duration' for realistic pre-call predictive model.")
else:
    print("Including 'duration' for benchmark performance.")

# Show numeric columns
numeric_df = df.select_dtypes(include=[np.number])
print("\n=== Numeric Columns Description ===")
print(numeric_df.describe(include='all'))

numeric_cols = numeric_df.columns
print("Numeric Columns:", list(numeric_cols))

# Show categorical columns
categorical_df = df.select_dtypes(include=[np.object_])
print("\n=== Categorical Columns Description ===")
print(categorical_df.describe(include='all'))

categorical_cols = categorical_df.columns
print("Categorical Columns:", list(categorical_cols))


Including 'duration' for benchmark performance.

=== Numeric Columns Description ===
               age      duration      campaign         pdays      previous  emp.var.rate  cons.price.idx  cons.conf.idx     euribor3m   nr.employed
count  41188.00000  41188.000000  41188.000000  41188.000000  41188.000000  41188.000000    41188.000000   41188.000000  41188.000000  41188.000000
mean      40.02406    258.285010      2.567593    962.475454      0.172963      0.081886       93.575664     -40.502600      3.621291   5167.035911
std       10.42125    259.279249      2.770014    186.910907      0.494901      1.570960        0.578840       4.628198      1.734447     72.251528
min       17.00000      0.000000      1.000000      0.000000      0.000000     -3.400000       92.201000     -50.800000      0.634000   4963.600000
25%       32.00000    102.000000      1.000000    999.000000      0.000000     -1.800000       93.075000     -42.700000      1.344000   5099.100000
50%       38.00000    180.0

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [23]:
# Goal is to predict if a client will subscirbe to a term deposit (column y: yes, no)
# This would allow the bank to focust on high potential leads improving marketing efficiency

In [24]:
# We need to understand how many times 'unknown' appears in each object column.
print("\n=== Checking 'unknown' for each object column ===")
for col in df.select_dtypes(include=['object']).columns:
    unknown_count = (df[col] == 'unknown').sum()
    if unknown_count > 0:
        print(f"Column '{col}' has {unknown_count} 'unknown' values.")
'''
Column 'job' has 330 'unknown' values.
Column 'marital' has 80 'unknown' values.
Column 'education' has 1731 'unknown' values.
Column 'default' has 8597 'unknown' values.
Column 'housing' has 990 'unknown' values.
Column 'loan' has 990 'unknown' values.
'''



=== Checking 'unknown' for each object column ===
Column 'job' has 330 'unknown' values.
Column 'marital' has 80 'unknown' values.
Column 'education' has 1731 'unknown' values.
Column 'default' has 8597 'unknown' values.
Column 'housing' has 990 'unknown' values.
Column 'loan' has 990 'unknown' values.


"\nColumn 'job' has 330 'unknown' values.\nColumn 'marital' has 80 'unknown' values.\nColumn 'education' has 1731 'unknown' values.\nColumn 'default' has 8597 'unknown' values.\nColumn 'housing' has 990 'unknown' values.\nColumn 'loan' has 990 'unknown' values.\n"

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [25]:
X = df.drop('y', axis=1)  # all features except target
y = df['y']               # target is 'yes'/'no'

# Identify numeric vs. categorical columns for transformations
numeric_cols_for_preproc = [
    col for col in df.select_dtypes(include=[np.number]).columns 
    if col != 'y'
]
categorical_cols_for_preproc = [
    col for col in df.select_dtypes(include=['object']).columns
    if col != 'y'
]

print("\nNumeric columns used for preprocessing:", numeric_cols_for_preproc)
print("Categorical columns used for preprocessing:", categorical_cols_for_preproc)



Numeric columns used for preprocessing: ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
Categorical columns used for preprocessing: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [26]:
# Build a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols_for_preproc),
        ('cat', OneHotEncoder(drop='first'), categorical_cols_for_preproc)
    ],
    remainder='drop'
)


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)
print(f"\nTrain set: {X_train.shape}, Test set: {X_test.shape}")



Train set: (28831, 20), Test set: (12357, 20)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [28]:
# Define pipelines & param grids in a dictionary to iterate.
#(k-NN, Logistic Regression, Decision Tree, SVM)
pipelines = {}



In [29]:
# kNN
knn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier())
])
knn_param_grid = {
    'knn__n_neighbors': [3, 5, 7, 11],
    'knn__weights': ['uniform', 'distance']
}
pipelines['KNN'] = (knn_pipeline, knn_param_grid)

# Logistic Regression
logreg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression(solver='saga', max_iter=1000))
])
logreg_param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],
    'logreg__penalty': ['l1','l2']  # saga supports both
}
pipelines['LogisticRegression'] = (logreg_pipeline, logreg_param_grid)

# Decision Tree
tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeClassifier(random_state=42))
])
tree_param_grid = {
    'tree__criterion': ['gini','entropy'],
    'tree__max_depth': [5, 10, 20, None],
    'tree__min_samples_split': [2, 5, 10],
    'tree__min_samples_leaf': [1, 2, 5]
}
pipelines['DecisionTree'] = (tree_pipeline, tree_param_grid)


# SVM
svm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svm', SVC(probability=True))
])
svm_param_grid = {
    'svm__kernel': ['linear','rbf'],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale','auto']
}
pipelines['SVM'] = (svm_pipeline, svm_param_grid)



### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

### Problem 9: Score the Model

What is the accuracy of your model?

In [31]:
f1_yes_scorer = make_scorer(f1_score, pos_label='yes')


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [32]:
import time
model_results = {}
comparison_table = []  # We'll store (model, train_time, train_accuracy, test_accuracy)

for model_name, (model_pipeline, param_grid) in pipelines.items():
    print(f"\n==== Now tuning hyperparameters for {model_name} ====")
    
    # GridSearchCV with F1 (pos_label='yes')
    grid_search = GridSearchCV(
        estimator=model_pipeline,
        param_grid=param_grid,
        scoring=f1_yes_scorer,
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    
    # Track training (fit) time
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()
    
    train_time = end_time - start_time
    
    best_model = grid_search.best_estimator_
    # Predictions on test set
    test_preds = best_model.predict(X_test)
    # Predictions on training set (to see train accuracy)
    train_preds = best_model.predict(X_train)
    
    # Evaluate
    test_acc = accuracy_score(y_test, test_preds)
    train_acc = accuracy_score(y_train, train_preds)
    test_f1 = f1_score(y_test, test_preds, pos_label='yes')
    train_f1 = f1_score(y_train, train_preds, pos_label='yes')
    
    print(f"Best Params for {model_name}:", grid_search.best_params_)
    print(f"Test F1: {test_f1:.4f}, Test Accuracy: {test_acc:.4f}")
    
    # Store detailed results
    model_results[model_name] = {
        'best_model': best_model,
        'best_params': grid_search.best_params_,
        'test_preds': test_preds,
        'train_preds': train_preds,
        'test_accuracy': test_acc,
        'train_accuracy': train_acc,
        'test_f1': test_f1,
        'train_f1': train_f1,
        'train_time': train_time
    }
    
    # For a quick comparison table
    comparison_table.append({
        'Model': model_name,
        'Train Time (s)': train_time,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })

# Present findings in a DataFrame
comparison_df = pd.DataFrame(comparison_table)
print("\n=== MODEL COMPARISON TABLE ===")
print(comparison_df)





==== Now tuning hyperparameters for KNN ====
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Params for KNN: {'knn__n_neighbors': 7, 'knn__weights': 'distance'}
Test F1: 0.4938, Test Accuracy: 0.9044

==== Now tuning hyperparameters for LogisticRegression ====
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Params for LogisticRegression: {'logreg__C': 10, 'logreg__penalty': 'l1'}
Test F1: 0.5233, Test Accuracy: 0.9123

==== Now tuning hyperparameters for DecisionTree ====
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Params for DecisionTree: {'tree__criterion': 'gini', 'tree__max_depth': 5, 'tree__min_samples_leaf': 1, 'tree__min_samples_split': 2}
Test F1: 0.5707, Test Accuracy: 0.9150

==== Now tuning hyperparameters for SVM ====
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/jimlopez/.pyenv/versions/3.13.0/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Params for SVM: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Test F1: 0.5391, Test Accuracy: 0.9099

=== MODEL COMPARISON TABLE ===
                Model  Train Time (s)  Train Accuracy  Test Accuracy
0                 KNN        5.602847        1.000000       0.904427
1  LogisticRegression       29.540414        0.910999       0.912276
2        DecisionTree        8.123339        0.916687       0.915028
3                 SVM     2029.245218        0.954112       0.909930


In [ ]:
'''
==== Now tuning hyperparameters for KNN ====
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Params for KNN: {'knn__n_neighbors': 7, 'knn__weights': 'distance'}
Test F1: 0.4938, Test Accuracy: 0.9044

==== Now tuning hyperparameters for LogisticRegression ====
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Params for LogisticRegression: {'logreg__C': 10, 'logreg__penalty': 'l1'}
Test F1: 0.5233, Test Accuracy: 0.9123

==== Now tuning hyperparameters for DecisionTree ====
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Params for DecisionTree: {'tree__criterion': 'gini', 'tree__max_depth': 5, 'tree__min_samples_leaf': 1, 'tree__min_samples_split': 2}
Test F1: 0.5707, Test Accuracy: 0.9150

==== Now tuning hyperparameters for SVM ====
Fitting 5 folds for each of 16 candidates, totalling 80 fits

Best Params for SVM: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Test F1: 0.5391, Test Accuracy: 0.9099

=== MODEL COMPARISON TABLE ===
                Model  Train Time (s)  Train Accuracy  Test Accuracy
0                 KNN        5.602847        1.000000       0.904427
1  LogisticRegression       29.540414        0.910999       0.912276
2        DecisionTree        8.123339        0.916687       0.915028
3                 SVM     2029.245218        0.954112       0.909930  

'''

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [33]:
# Printing final classification reports for each model
for model_name, info in model_results.items():
    print(f"\n=== Detailed Evaluation for {model_name} ===")
    print("Best Params:", info['best_params'])
    
    # Classification report on Test set
    print("Classification Report (Test):")
    print(classification_report(y_test, info['test_preds']))
    
    # Confusion matrix
    print("Confusion Matrix (Test):")
    print(confusion_matrix(y_test, info['test_preds']))


print("\nAll done! We have baseline + tuned models (KNN, LogisticRegression, DecisionTree, SVM) and a comparison table. Feel free to expand.")


=== Detailed Evaluation for KNN ===
Best Params: {'knn__n_neighbors': 7, 'knn__weights': 'distance'}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.93      0.97      0.95     10965
         yes       0.61      0.41      0.49      1392

    accuracy                           0.90     12357
   macro avg       0.77      0.69      0.72     12357
weighted avg       0.89      0.90      0.90     12357

Confusion Matrix (Test):
[[10600   365]
 [  816   576]]

=== Detailed Evaluation for LogisticRegression ===
Best Params: {'logreg__C': 10, 'logreg__penalty': 'l1'}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.93      0.97      0.95     10965
         yes       0.67      0.43      0.52      1392

    accuracy                           0.91     12357
   macro avg       0.80      0.70      0.74     12357
weighted avg       0.90      0.91      0.90     12357

Confusion Matrix (Te

In [ ]:
'''
=== Detailed Evaluation for KNN ===
Best Params: {'knn__n_neighbors': 7, 'knn__weights': 'distance'}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.93      0.97      0.95     10965
         yes       0.61      0.41      0.49      1392

    accuracy                           0.90     12357
   macro avg       0.77      0.69      0.72     12357
weighted avg       0.89      0.90      0.90     12357

Confusion Matrix (Test):
[[10600   365]
 [  816   576]]

=== Detailed Evaluation for LogisticRegression ===
Best Params: {'logreg__C': 10, 'logreg__penalty': 'l1'}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.93      0.97      0.95     10965
         yes       0.67      0.43      0.52      1392

    accuracy                           0.91     12357
   macro avg       0.80      0.70      0.74     12357
weighted avg       0.90      0.91      0.90     12357

Confusion Matrix (Test):
[[10678   287]
 [  797   595]]

=== Detailed Evaluation for DecisionTree ===
Best Params: {'tree__criterion': 'gini', 'tree__max_depth': 5, 'tree__min_samples_leaf': 1, 'tree__min_samples_split': 2}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.94      0.97      0.95     10965
         yes       0.66      0.50      0.57      1392

    accuracy                           0.92     12357
   macro avg       0.80      0.73      0.76     12357
weighted avg       0.91      0.92      0.91     12357

Confusion Matrix (Test):
[[10609   356]
 [  694   698]]

=== Detailed Evaluation for SVM ===
Best Params: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Classification Report (Test):
              precision    recall  f1-score   support

          no       0.93      0.97      0.95     10965
         yes       0.64      0.47      0.54      1392

    accuracy                           0.91     12357
   macro avg       0.79      0.72      0.74     12357
weighted avg       0.90      0.91      0.90     12357

Confusion Matrix (Test):
[[10593   372]
 [  741   651]]

All done! We have baseline + tuned models (KNN, LogisticRegression, DecisionTree, SVM) and a comparison table. Feel free to expand.'''

##### Questions